In [1]:
import torch
import torch.nn as nn

## w/o LSTM | Univariate case

In [2]:
# source: https://pytorch.org/tutorials/intermediate/char_rnn_generation_tutorial.html

In [4]:
# Dummy data
seq = [i for i in range(5, 100, 5)]

def prepare_seq(seq, n_steps=3):
    samples, targets = [], []
    start_i, end_i = 0, 0 + n_steps
    while end_i <= len(seq) - 1:
        sample, target = seq[start_i:end_i], seq[end_i]         
        samples.append(sample)
        targets.append(target)
        start_i += 1
        end_i += 1
    return torch.Tensor(samples), torch.Tensor(targets)

samples, targets = prepare_seq(seq)

for sample, target in zip(samples, targets):
    print(sample, target)

tensor([ 5., 10., 15.]) tensor(20.)
tensor([10., 15., 20.]) tensor(25.)
tensor([15., 20., 25.]) tensor(30.)
tensor([20., 25., 30.]) tensor(35.)
tensor([25., 30., 35.]) tensor(40.)
tensor([30., 35., 40.]) tensor(45.)
tensor([35., 40., 45.]) tensor(50.)
tensor([40., 45., 50.]) tensor(55.)
tensor([45., 50., 55.]) tensor(60.)
tensor([50., 55., 60.]) tensor(65.)
tensor([55., 60., 65.]) tensor(70.)
tensor([60., 65., 70.]) tensor(75.)
tensor([65., 70., 75.]) tensor(80.)
tensor([70., 75., 80.]) tensor(85.)
tensor([75., 80., 85.]) tensor(90.)
tensor([80., 85., 90.]) tensor(95.)


In [6]:
#samples = sample.reshape()

In [22]:
class RNN(nn.Module):
    def __init__(self, in_size, hidden_size, out_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.fc1 = nn.Linear(in_size + hidden_size, out_size)
        self.fc_hidden = nn.Linear(in_size + hidden_size, hidden_size)
        #self.fc_2 = nn.Linear(out_size + hidden_size, out_size)  # 
    
    def forward(self, inputs, hidden):
        print(inputs.shape)
        print(hidden.shape)
        combined_inputs = torch.cat((inputs, hidden), 1)
        hidden = self.hidden(combined_inputs)
        output = self.fc1(combined_inputs)
        return output, hidden
        
    def init_hidden(self):
        return torch.zeros(self.hidden_size)         

In [23]:
loss_fn = nn.MSELoss()
lr = 0.0001


def train(model, inputs, target):
    hidden = model.init_hidden()
    model.zero_grad()
    
    loss_total = 0
    
    for i in range(inputs.shape[0]):
        output, hidden = model(inputs[i], hidden)
        loss = loss_fn(output, target)
        loss_total += loss
    
    loss.backward()
                   
    for p in model.parameters():
        p.data.add_(-lr, p.grad.data)

    return output, loss_total.item() / inputs.shape[0]

In [24]:
model = RNN(3, 128, 1)

n_iters = 10
print_every = 5
#plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

for it in range(1, n_iters + 1):
    output, loss = train(model, samples, targets)
    total_loss += loss

    if it % print_every == 0:
        print('%s (%d %d%%) %.4f' % (it, it / n_iters * 100, loss))

torch.Size([3])
torch.Size([128])


RuntimeError: Dimension out of range (expected to be in range of [-1, 0], but got 1)